<a href="https://colab.research.google.com/github/majdhammami/Prostate-cANcer-graDe-Assessment/blob/main/Keras_VGG16_VGG19_InceptionV3_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'prostate-cancer-grade-assessment:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F18647%2F1126921%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T213624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D87b56250da9718e61d54c7cea864f74932b9678626142014b0d1e11e0a653abd25cfc13247c3b84c414491257adcb477d53fa682f117913b781fa5fe4e14f80b67339d5ff58088f5f306431ffd2a78906c107298c4673cbd109d1ac50f13a62de0590b67ab6409deb93db215834f858711780e948052d5dd5e36188022353af6c8b02dd6c044e613742cf501594f2b132db71a388ac0956c36fca1e2744c3ccba6fb4a42947820d7651c6ea7d45e6d8dd7fa80c9be624bd1720c68fac01fb8d3a8e0551fefb32ad980d347f45abb275204adade5b8f635ce70a034a907a9f4b2f2bffcb3e0a8db8402fdadeed50b8166c06559020a9b78c192b9d0ad21859e62,keras-pretrained-models:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2798%2F7251%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T213624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9c03f33aae2e34af7dcd3f2d10eb283d4c70751c79b20417f011a11cdbf5b07955b9453ec25ab01f61ac5082687ddc64754b8de8d71e6fcd216e55f02c8fe82afc1232f0a7c2819cb234b0089956a1e13a945f97446cbc2484dbc9474a8169cbe02f11a2366e86dd743b590a4443532155a90ebf4be4397a734b568bc19b52b7788a871f71c19106d936ff75b1eaafdac5904cd5fbbcf8b83c1a81073d4a2d403680053ce45e8a77cd0a1f7ecc461063b451252d56d9eb98474efce7fadded025079cd392b28e5529f1a26b6ed7a128bfcbb1e7e37fdc2bd1d93e52f61eef60e437a5bd5d1f66822f3096585b24f3091422d9daf93083c3af5ff8841a4ac9147,vgg19:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F6303%2F9897%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T213624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D37ac24f1e2a1b6b2506bacc521669dcfbbc210afbfbdc6df2f0bd59d045413a47cc721c702fbac43fbdd8961138bc23b4fe7b2777968ad3a5eb2cfd72e7e5302128c6cdbc44263a533ac6c2432747b9782751ec43082ce3c1bfe3d9c571f16ab4e3d2ba87c0a747713ea5675b5607907ef452c52a5624e740edebb2bac31c808345aa72492d0535240c132125bdb87259a4c8b5aa6c010be8e4cfa73d90adf1b0518abb6846b1599a02f8ba2f8197adc40db9ba9b39db0ba2f5db7bee966113dbfcc130065d297ee987099af616e773a0d7554c0e3ae2a62986541e5028fe9d7c4dc5fb586d682631eb8cb6ad295896a4ae887230fd83c4cde660510e5a5a042,inceptionv3:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F6306%2F9901%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T213624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D44508b60f8844d063841c94e6a00dfe737cbe9929086070d72e7fd642c94bb6266b670cce20ba9d0120d079c13da21bfb8a048be1cc60a63b23d2a07e84af94fadfb87de41919a6226487a0ffe79f22e23ad2d44658f098d45b52fde230a14efa90f2b23ff0f3927a7c6beacb9a9d6f2edb6deb7236b4e843d40ecc2bdc1f95e114920fca594869c7218c2c246bf20ec1fed0055a6e54282c02cb53ef031f6cdeffe79ed4f73a0dcae6fafda1fd30fde379ee3fe1b46c87c9ea0fc7a125d2b2c383bda819e0ac5f5653f8e6fc3d86b18762c6ea743a9193cf36fa1d889486bf1f28fd1e3760538b9475352e41587d405ea84740ebba4d4dc449d11cd5c36e03a,panda-resized-train-data-512x512:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F615046%2F1101206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T213624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0a72fbb8c8bfacbfe17bff990876704b28e1a49f3df64cad71be1d61d916141000ead86a16350edf8deb27ad5bc54fd92690fadd8a343f72f95d22956a991968f40fd3cf1c716e8de45f8c65c49ec594f38dc6e821a7a46d3a99d5ef0174ffb250cffb8504bc4e0a01bdbb48841c262ad4f81dfa3b53cdf86a081f16361e1e8b050fdd5a8ba506d32c5e8d9321c8b2625a1788ee1b7b5dfe3bf1fc479555da5d2cf37fccb255d5dfd8269620a78f96e00b65b29ad4c616ae0d18ff6ea6c677da35cf25a5f5224289ee3ec089fb821914a21308e605711f2fac87bcdc6120679c5b9ac69d6340ac5000ade7d21394b23a0f92fca12e16f71ff54a79b68832ee41,bbbbbb:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2110487%2F3506655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T213624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D14155180fc063b6ec708d90028c59b8a6071a857d866d9e212fe8ec9ff89ad196991a9984416eeb6307d495260c0113290dd3609574574d8886a88c3322b22bf21f338347aa695d6395dbc8242573a798565fd4cdc91e22e871ce5ab4380db6b169af24e92eb81885af50bdc9d0876540fbbd189a6d11cce5014c49e6b8da77e5f494ae4a83c2b1e9051bcc6d0aab934b5cdfcb9b1f2fc536b391ea81e13512aa038f27be511a387dc50563d99e7be6767356d5bcad024687d9cdb2a76ccfb17750f0f94c2bf0e3da05eb082c36d0ac0a33958b2e928cbe1a81d9fe374de4c7c78e3c1ed679b411f2617df2cf93f39c4a7f9b8cdfa565fc1b757b082c0cb4d56'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


VGG16 Baseline

Data-
* train.csv:- image_id,karlonski/radhound, gleason and ISUP grade.
* test.csv:-  image_ids and data_provider.
* train_images:- A folder containing biopsy images in a large multilevel .tiff file, some of them will be used for testing).
* train_label_masks:-Folder with segmentation masks showing which parts of the image led to the ISUP grade. Not all training images have label masks, and there may be false positives or false negatives in the label masks. These masks may assist with the development of strategies for selecting useful image subsamples and the values might depend also on the data provider.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import openslide
import os
import cv2
import PIL
from IPython.display import Image, display
from keras.applications.vgg16 import VGG16,preprocess_input
# Plotly for the interactive viewer (see last section)
import plotly.graph_objs as go
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import gc
import matplotlib.pyplot as plt
%matplotlib inline

import skimage.io
from sklearn.model_selection import KFold
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import tensorflow as tf
from tensorflow.python.keras import backend as K
sess = K.get_session()

In [ ]:
from keras.callbacks.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.applications.xception import Xception
import time
import seaborn as sns
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:

from numpy.random import seed
seed(1)

In [ ]:
train=pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
train.head()
train_copy = train.copy()


Let's take a look at one of the biopsy.

In [ ]:
img=openslide.OpenSlide('/kaggle/input/prostate-cancer-grade-assessment/train_images/2fd1c7dc4a0f3a546a59717d8e9d28c3.tiff')
display(img.get_thumbnail(size=(512,512)))

In [ ]:
img.dimensions

These images are pretty big in size for training I'll be using the resized 512x512 dataset uploaded by @xhlulu.

In [ ]:
patch = img.read_region((18500,4100), 0, (256, 256))

# Display the image
display(patch)
# Close the opened slide after use
img.close()

We are asked to predict ISUP grade(scale 0-5) for each prostate biopsy image, which is in turn derived from the gleason score(three categories 3,4,5).Before diving deeper into the competition let's try to understand these first.

## Gleason Score:-
Since prostate tumors are often made up of cancerous cells that have different grades, two grades are assigned for each patient.  A primary grade is given to describe the cells that make up the largest area of the tumor and a secondary grade is given to describe the cells of the next largest area.  For instance, if the Gleason Score is written as 3+4=7, it means most of the tumor is grade 3 and the next largest section of the tumor is grade 4, together they make up the total Gleason Score.  If the cancer is almost entirely made up of cells with the same score, the grade for that area is counted twice to calculated the total Gleason Score.

The samples are made up of glandular tissue and connective tissue. The glands are hollow structures, which can be seen as white “holes” or branched cavities in the WSI. The appearance of the glands forms the basis of the Gleason grading system.

![](https://www.prostateconditions.org/images/about/murtagh7e_c114_f04-2.png)


This is how Gleason score is decided from(3-5), 5 being the most severe. To understand more [read this](https://www.kaggle.com/c/prostate-cancer-grade-assessment/overview/additional-resources).


## ISUP grade
According to current guidelines by the International Society of Urological Pathology (ISUP), the Gleason scores are summarized into an ISUP grade on a scale from 1 to 5 according to the following rule:

* Gleason score 6 = ISUP grade 1
* Gleason score 7 (3 + 4) = ISUP grade 2
* Gleason score 7 (4 + 3) = ISUP grade 3
* Gleason score 8 = ISUP grade 4
* Gleason score 9-10 = ISUP grade 5.

An example as provided on the official page of the competition can make us understand better.

![](https://storage.googleapis.com/kaggle-media/competitions/PANDA/Screen%20Shot%202020-04-08%20at%202.03.53%20PM.png)



The most common (blue outline, Gleason pattern 3) and second most common (red outline, Gleason pattern 4) cancer growth patterns present in the biopsy dictate the Gleason score (3+4 for this biopsy), which in turn is converted into an ISUP grade (2 for this biopsy) following guidelines of the International Society of Urological Pathology. Biopsies not containing cancer are represented by an ISUP grade of 0 in this challenge.

Let's also look at the class distribution.

In [ ]:
train['isup_grade'].value_counts()

The classes are imbalanced, more severe cases are underrepresented.

Let's start preparing the images for training.

In [ ]:
train.head()

In [ ]:
labels=[]
data=[]
data_dir='/kaggle/input/panda-resized-train-data-512x512/train_images/train_images/'
for i in range(train.shape[0]):
    data.append(data_dir + train['image_id'].iloc[i]+'.png')
    labels.append(train['isup_grade'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['isup_grade']=labels

In [ ]:
df.head()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df['images'],df['isup_grade'], test_size=0.2, random_state=42)


In [ ]:
train=pd.DataFrame(X_train)
train.columns=['images']
train['isup_grade']=y_train

validation=pd.DataFrame(X_val)
validation.columns=['images']
validation['isup_grade']=y_val

train['isup_grade']=train['isup_grade'].astype(str)
validation['isup_grade']=validation['isup_grade'].astype(str)

So the basic preprocessing I've done is:-
* Normalizing the images.
* Reshape the images to be of shape 224,224,3
* Basic image augmentation like rotation, flipping etc.

In [ ]:
#Image data generator generates varied images with the input data for better prediction for the models
#shear is avoided since the cancerous cells look like sheared normal cells

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,
    featurewise_center=True,
    featurewise_std_normalization=True,
    #zoom_range=[0.8, 1.2],
    horizontal_flip=True, vertical_flip = True,
    brightness_range=[0.9, 1.1],
    width_shift_range=1.0,
    height_shift_range=1.0)#,
    #validation_split=0.1)


val_datagen=train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='images',
    y_col='isup_grade',
    target_size=(224, 224),
    batch_size=32,
    shuffle = True,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
    validation,
    x_col='images',
    y_col='isup_grade',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
filenames = validation_generator.filenames
nb_samples = len(filenames)



In [ ]:
y_true = validation_generator.classes



In [ ]:
#y_true

In [ ]:
#VGG16 model

In [ ]:
def vgg16_model( num_classes=None):

    model = VGG16(weights='/kaggle/input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
    #x=Dropout(0.2)(model.output)
    x=Flatten()(model.output)
    #x =Dense(32, activation = 'relu')(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model

#vgg_conv=vgg16_model(6)

In [ ]:
#VGG19 model-- Uncomment last line to run the model

In [ ]:
from keras.applications.vgg19 import VGG19
def vgg19_model(num_classes = None):
    #vgg19_weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
    model = VGG19(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
    #model = VGG19(weights='/input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels.h5', include_top=False, input_shape=(224, 224, 3))
    x=Dropout(0.3)(model.output)
    x=Flatten()(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
#vgg19_conv = vgg19_model(6)

In [ ]:
#InceptionV3 model- Uncomment last line to run the model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
def InceptionV3_model(num_classes = None):
    InceptionV3_weights = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = ResNet50(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = InceptionV3(weights= InceptionV3_weights, include_top=False, input_shape=(224, 224, 3))
    #model = InceptionV3(weights='imagenet', include_top = False, input_shape = (224,224,3))
    x=Dropout(0.3)(model.output)
    #x=Flatten()(model.output)
    #x =Dense(64, activation = 'relu')(model.output)
    #x = tf.compat.v1.keras.layers.GlobalAveragePooling2D()(model.output)
    x=Flatten()(x)
    #x =Dropout(0.2)(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
InceptionV3_conv = InceptionV3_model(6)

In [ ]:
#Resnet50 model- Uncomment last line to run the model

In [ ]:
#!pip install tensorflow==2.1.0

In [ ]:
from keras.applications.resnet50 import ResNet50
def ResNet50_model(num_classes = None):
    ResNet_weights = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = ResNet50(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = ResNet50(weights= ResNet_weights, include_top=False, input_shape=(224, 224, 3))
    #x=Dropout(0.2)(model.output)
    #x = GlobalAveragePooling2D()(model.output)
    x=Flatten()(model.output)
    #x =Dropout(0.2)(x)
    x =Dense(16, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
#ResNet50_conv = ResNet50_model(6)

In [ ]:
from keras.applications import InceptionResNetV2
def InceptionResnet_model(num_classes = None):
    InceptionResnet_weights = '../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = inception_resnet_v2(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = InceptionResNetV2(weights= InceptionResnet_weights, include_top=False, input_shape=(224, 224, 3))
    #x=Dropout(0.2)(model.output)
    #x = GlobalAveragePooling2D()(model.output)
    x=Flatten()(model.output)
    #x =Dropout(0.2)(x)
    x =Dense(16, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
#InceptionResnet_conv = InceptionResnet_model(6)

Let's take a look at the architecture.

In [ ]:
#Uncomment any one to run the corresponding summary

In [ ]:
#vgg_conv.summary()

In [ ]:
#vgg19_conv.summary()

In [ ]:

InceptionV3_conv.summary()

In [ ]:
#ResNet50_conv.summary()

In [ ]:
#InceptionResnet_conv.summary()

In [ ]:
#Including batch Normalization

In [ ]:
def freeze(model):
    # freeze layers before 99
    for layer in model.layers[:99]:
        layer.trainable = False
        if layer.name.startswith('batch_normalization'):
            layer.trainable = True
        if layer.name.endswith('bn'):
            layer.trainable = True

    # unfreeze layers after 99
    for layer in model.layers[99:]:
        layer.trainable = True






In [ ]:
#freeze(vgg_conv)
#freeze(vgg19_conv)
freeze(InceptionV3_conv)
#freeze(ResNet50_conv)

In [ ]:
y_pred = np.argmax(InceptionV3_conv.predict_generator(validation_generator, steps= len(validation_generator)), axis=1)

In [ ]:
#kappa score is the metric we use since two types of evaluation is done on the data set

#The kappa statistic.
#According to Cohen's original article,
#values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight,
#0.21–0.40 as fair, 0.41– 0.60 as moderate,
#0.61–0.80 as substantial,
#0.81–1.00 as almost perfect agreement.

In [ ]:
def kappa_score(y_true, y_pred):

    y_true=tf.math.argmax(y_true)
    y_pred=tf.math.argmax(y_pred)
    #return tf.compat.v1.py_func(cohen_kappa_score,(y_true, y_pred),tf.double)
    return tf.compat.v1.py_func(cohen_kappa_score,(y_true,y_pred),tf.double)
    #return (y_true,y_pred)

In [ ]:
import numpy as np
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

import keras.backend as K
import tensorflow as tf


def kappa_keras(y_true, y_pred):

    y_true = K.cast(K.argmax(y_true, axis=-1), dtype='int32')
    y_pred = K.cast(K.argmax(y_pred, axis=-1), dtype='int32')
    #print(y_true)
    #print(y_pred)
    # Figure out normalized expected values
    min_rating = K.minimum(K.min(y_true), K.min(y_pred))
    max_rating = K.maximum(K.max(y_true), K.max(y_pred))

    # shift the values so that the lowest value is 0
    # (to support scales that include negative values)
    y_true = K.map_fn(lambda y: y - min_rating, y_true, dtype='int32')
    y_pred = K.map_fn(lambda y: y - min_rating, y_pred, dtype='int32')

    # Build the observed/confusion matrix
    num_ratings = max_rating - min_rating + 1
    observed = tf.math.confusion_matrix(y_true, y_pred,
                                num_classes=num_ratings)
    num_scored_items = K.shape(y_true)[0]

    weights = K.expand_dims(K.arange(num_ratings), axis=-1) - K.expand_dims(K.arange(num_ratings), axis=0)
    weights = K.cast(K.pow(weights, 2), dtype='float64')

    hist_true = tf.math.bincount(y_true, minlength=num_ratings)
    hist_true = hist_true[:num_ratings] / num_scored_items
    hist_pred = tf.math.bincount(y_pred, minlength=num_ratings)
    hist_pred = hist_pred[:num_ratings] / num_scored_items
    expected = K.dot(K.expand_dims(hist_true, axis=-1), K.expand_dims(hist_pred, axis=0))

    # Normalize observed array
    observed = observed / num_scored_items

    # If all weights are zero, that means no disagreements matter.
    score = tf.where(K.any(K.not_equal(weights, 0)),
                     K.sum(weights * observed) / K.sum(weights * expected),
                     0)

    return 1. - score

if __name__ == '__main__':
    y_true = np.array([2, 0, 2, 2, 0, 1])
    y_pred = np.array([0, 0, 2, 2, 0, 2])
    # Testing Keras implementation of QWK

    # Calculating QWK score with scikit-learn

    skl_score = cohen_kappa_score(y_true, y_pred, weights='quadratic')

    # Keras implementation of QWK work with one hot encoding labels and predictions (also it works with softmax probabilities)
    # Converting arrays to one hot encoded representation
    shape = (y_true.shape[0], np.maximum(y_true.max(), y_pred.max()) + 1)

    y_true_ohe = np.zeros(shape)
    y_true_ohe[np.arange(shape[0]), y_true] = 1

    y_pred_ohe = np.zeros(shape)
    y_pred_ohe[np.arange(shape[0]), y_pred] = 1

    # Calculating QWK score with Keras
    with tf.compat.v1.Session() as sess:
        keras_score = kappa_keras(y_true_ohe, y_pred_ohe).eval()

    #print('Scikit-learn score: {:.03}, Keras score: {:.03}'.format(skl_score, keras_score))


Hyperparameter Tuning

In [ ]:
opt = SGD(lr= 0.0005, momentum=0.9,decay=1e-4)
#vgg_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=[kappa_keras, 'accuracy'])
#vgg19_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=[kappa_keras, 'accuracy'])
InceptionV3_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=[kappa_score,kappa_keras,'accuracy'])
#ResNet50_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=[kappa_score,kappa_keras, 'accuracy'])
#InceptionResnet_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=[kappa_score,kappa_keras, 'accuracy'])

In [ ]:
nb_epochs = 4
batch_size=32
nb_train_steps = train.shape[0]//batch_size
nb_val_steps=validation.shape[0]//batch_size
#nb_train_steps = 128
#nb_val_steps = 64
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
#vgg_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=10,validation_data=validation_generator,
#validation_steps=nb_val_steps),


In [ ]:
#generator to activate the augmentation.
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5),
             EarlyStopping(monitor='val_loss', patience=1),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
#history =vgg19_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=5,validation_data=validation_generator,validation_steps=nb_val_steps,callbacks = callbacks)

In [ ]:
#history=ResNet50_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=20,validation_data=validation_generator,
#validation_steps=nb_val_steps, callbacks = callbacks)

In [ ]:
history = InceptionV3_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=30,validation_data=validation_generator,
validation_steps=nb_val_steps, callbacks = callbacks)

In [ ]:
#history = ptionResnet_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=2,validation_data=validation_generator,
#validation_steps=nb_val_steps, callbacks = callbacks)

In [ ]:
def show_history(history):
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('kappa_keras')
    ax[1].plot(history.epoch, history.history["kappa_keras"], label="Train Quadratic Kappa score")
    ax[1].plot(history.epoch, history.history["val_kappa_keras"], label="Validation Quadratic Kappa score")
    ax[2].set_title('accuracy')
    ax[2].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[2].plot(history.epoch, history.history["val_accuracy"], label="Validation accuracy")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

In [ ]:
show_history(history)

In [ ]:
#vgg_baseline = Model.save('VGG16_Baseline.h5')  # creates a HDF5 file 'my_model.h5'
#InceptionV3_baseline =InceptionV3_conv.save('InceptionV3_Baseline.h5')  # creates a HDF5 file 'my_model.h5'
#Resnet50_baseline =ResNet50_conv.save('ResNet50_Baseline.h5')  # creates a HDF5 file 'my_model.h5'
#InceptionResnet_baseline =InceptionResnet_conv.save('InceptionResnet_Baseline.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
#vgg_baseline_weights = Model.save_weights('vgg_baseline_weights.h5')
#InceptionV3_weights =InceptionV3_conv.save_weights('InceptionV3_weights.h5')  # creates a HDF5 file 'my_model.h5'
#Resnet50_weights =ResNet50_conv.save_weights('Resnet50_weights.h5')  # creates a HDF5 file 'my_model.h5'
#InceptionResnet_weights =InceptionResnet_conv.save_weights('InceptionResnet_weights.h5')  # creates a HDF5 file 'my_model.h5'

Inference


In [ ]:
os.listdir('.')

In [ ]:
#InceptionV3_conv.load_weights("best_model.h5")
#InceptionResnet_conv.load_weights("best_model.h5")

In [ ]:
def predict_submission(df, path, passes=1):

    df["image_path"] = [path+image_id+".tiff" for image_id in df["image_id"]]
    df["isup_grade"] = 0

    for idx, row in df.iterrows():
        prediction_per_pass = []
        for i in range(passes):
            model_input = np.array([get_random_samples(row.image_path)/255.])
            input_image1 = model_input[:,0,:,:]
            input_image2 = model_input[:,1,:,:]
            input_image3 = model_input[:,2,:,:]

            prediction = InceptionV3_conv.predict([input_image1,input_image2,input_image3])
            prediction_per_pass.append(np.argmax(prediction))

        df.at[idx,"isup_grade"] = np.mean(prediction_per_pass)
    df = df.drop('image_path', 1)
    return df[["image_id","isup_grade"]]

In [ ]:
# submission code from https://www.kaggle.com/frlemarchand/high-res-samples-into-multi-input-cnn-keras
def predict_submission(df, path):

    df["image_path"] = [path+image_id+".tiff" for image_id in df["image_id"]]
    df["isup_grade"] = 0
    predictions = []
    for idx, row in df.iterrows():
        print(row.image_path)
        img=skimage.io.imread(str(row.image_path))
        img = cv2.resize(img, (224,224))
        img = cv2.resize(img, (224,224))
        img = img.astype(np.float32)/255.
        img=np.reshape(img,(1,224,224,3))


        prediction=InceptionV3_conv.predict(img)
        predictions.append(np.argmax(prediction))

    df["isup_grade"] = predictions
    df = df.drop('image_path', 1)
    return df[["image_id","isup_grade"]]

In [ ]:
test_path = "../input/prostate-cancer-grade-assessment/train_images/"
submission_df = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")

if os.path.exists(test_path):
    test_df = pd.read_csv("../input/bbbbbb/1.csv")
    submission_df = predict_submission(test_df, test_path)
else:
    print('submission csv not found')



submission_df.to_csv('submission.csv', index=False)
submission_df

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report,auc,roc_curve,cohen_kappa_score

In [ ]:
test_df["image_path"] = [path+image_id+".tiff" for image_id in test_df["image_id"]]
test_df["isup_grade"] = 0
predictions = []
for idx, row in df.iterrows():
        print(row.image_path)
        img=skimage.io.imread(str(row.image_path))
        img = cv2.resize(img, (224,224))
        img = cv2.resize(img, (224,224))
        img = img.astype(np.float32)/255.
        img=np.reshape(img,(1,224,224,3))


        prediction=InceptionV3_conv.predict(img)
        predictions.append(np.argmax(prediction))

test_df["isup_grade"] = predictions
test_df = df.drop('image_path', 1)

In [ ]:
preds= []
for i in X_val:
    img=skimage.io.imread(i)
    img = cv2.resize(img, (224,224))
    img = cv2.resize(img, (224,224))
    img = img.astype(np.float32)/255.
    img=np.reshape(img,(1,224,224,3))

    prediction=InceptionV3_conv.predict(img)
    preds.append(np.argmax(prediction))


accuracy_score(y_val,preds)

In [ ]:
print("accuracy_score:",accuracy_score(y_val,preds))

In [ ]:
confusion_matrix(y_val,preds,normalize='true')

In [ ]:
f,ax=plt.subplots(1,1,figsize=(12,10))
sns.heatmap(confusion_matrix(y_val,preds,normalize='true'),annot=True,fmt='2.3f')


In [ ]:
print(metrics.classification_report(y_val,preds))
